#### Write Vie SPAM Idea Dataset to CSV

In [ ]:
!pip3 install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Specify the file path
file_path = "data/vie/spam.csv"

# Write Vie Spam Dataset to CSV
for sent in en_sent:
  sentence = "en: " + sent
  inputs = [sentence]
  outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids, max_length=512)
  vie_trans = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][4:]
  write_to_file(vie_trans,0, file_path)


#### Write Vie Valid Idea Description Dataset to CSV

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Specify the file path
file_path = "data/vie/valid_idea.csv"

# Write Vie Valid Idea Description Dataset to CSV
for sent in en_sent_1:
  sentence = "en: " + sent
  inputs = [sentence]
  outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids, max_length=512)
  vie_trans = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][4:]
  write_to_file(vie_trans,1,file_path)

## Filtering by LSTM

### Import

In [ ]:
import pandas as pd

invalid_0 = pd.read_csv("data/invalid_sentences.csv", delimiter=';')
invalid_1 = pd.read_csv("data/random_str.csv", delimiter=',')
semantic = pd.read_csv("data/valid_ideas.csv", delimiter=';', usecols=['sentence', 'spam'])

invalid_0.head()
invalid_1.head()
semantic.head()

frames = [invalid_0, semantic, invalid_1]
 
df = pd.concat(frames)

df.head()

### Sequence Model

Install Tensorflow on M1: https://medium.com/@sorenlind/tensorflow-with-gpu-support-on-apple-silicon-mac-with-homebrew-and-without-conda-miniforge-915b2f15425b

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip3 install -U tensorflow-macos tensorflow-metal
!pip3 install plotlib tf-models-official

In [ ]:
!pip3 show tensorflow
!pip3 install tensorflow

In [ ]:
import os
import shutil

import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')
tf.test.is_gpu_available()
# from official.nlp import optimization  # to create AdamW optimizer

tf.get_logger().setLevel('ERROR')

In [ ]:
!pip3 install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split

#Create test and train samples from one dataframe with pandas?
train_data, test_data = train_test_split(df, test_size=0.3)

In [ ]:
train_list = train_data.values.tolist()
# print(train_list)

## EMBEDDING LAYER

In [ ]:
!pip3 install pandas numpy seaborn matplotlib scikit-learn keras

In [ ]:
!pip3 install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
# tf.keras.optimizers.legacy.RMSprop
from keras.src.optimizers.legacy.rmsprop import RMSprop
from keras.src.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [ ]:
import pandas as pd

invalid_0 = pd.read_csv("data/invalid_sentences.csv", delimiter=';')
invalid_1 = pd.read_csv("data/random_str.csv", delimiter=',')
semantic = pd.read_csv("data/valid_ideas.csv", delimiter=';', usecols=['sentence', 'spam'])

invalid_0.head()
invalid_1.head()
semantic.head()

frames = [invalid_0, semantic, invalid_1]
 
df = pd.concat(frames)

df = df.reset_index()
# df[100:120]
# df.index.duplicated()
df.index
# df.loc[df.index.unique()]

In [ ]:
sns.countplot(df, x="spam")
plt.xlabel('Label')
plt.title('Number of ham (1) and spam (0) ideas')

In [ ]:
X = df.sentence
Y = df.spam
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words,50,input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1,name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
history = model.fit(sequences_matrix,Y_train,batch_size=128,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
word_idx = 10
prediction = model.predict(test_sequences_matrix[word_idx:word_idx+10])
print(prediction, X_test[word_idx:word_idx+10])

In [ ]:
def input(new_sentence, ai_model):
  new_sentence = new_sentence + "."
  test_sequences = tok.texts_to_sequences([new_sentence])
  test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=max_len)
  prediction = ai_model.predict(test_sequences_matrix)
  label = "spam"
  if prediction > 0.5:
    label = "innovative idea"
  print(prediction, new_sentence[:30] + '...')

In [ ]:

input("Nghi is building an AI app for tracking habit automatically that use voice-over to record your daily activities", model)
input("Origin recipes for this manificient coffee shop", model)
input("This is the new way to play guitar", model)
input("This is the new book", model)
input("This is the new AI model helping homework completion", model)
input("We are operating new AI model to auto complete homework", model)
input("Instagram is so popular that now everyone is using, scrolling til tired", model)

In [ ]:
input("SmartMirror Health Assistant: A cutting-edge innovation that combines a traditional mirror with advanced AI technology to create a personalized health monitoring system. The SmartMirror analyzes facial features and provides real-time health insights, such as heart rate, stress levels, and skin condition. It also offers personalized workout recommendations and nutrition tips based on individual goals. With voice control and integration with smart devices, the SmartMirror Health Assistant revolutionizes daily routines, empowers users to make informed health decisions, and promotes overall well-being in a seamless and interactive way.", model)

In [ ]:
input("Paperclip Organizer: A simple and practical solution for keeping paperclips organized and easily accessible. This compact organizer features separate compartments for different sizes and colors of paperclips, preventing them from getting lost or tangled. Its sleek design takes up minimal desk space, offering convenience and efficiency for everyday office tasks.", model)
input("A budget-friendly hotel chain providing comfortable accommodations, basic amenities, and convenient locations near popular attractions.", model)
input("A travel agency offering pre-packaged vacation deals to popular tourist destinations, with standard itineraries and accommodations.", model)

In [ ]:
docs = """
A travel agency that specializes in creating immersive cultural experiences for travelers
"""

In [ ]:
input(docs, model)

## PhoBERT Embedding - Vietnamese Spam Filtering

[Hugging Face - PhoBERT](https://huggingface.co/docs/transformers/model_doc/phobert)

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
line = "Tôi là sinh_viên trường đại_học Công_nghệ ."

input_ids = torch.tensor([tokenizer.encode(line)])

with torch.no_grad():
  features = phobert(input_ids) # Models outputs are now tuples


In [ ]:
!pip3 install underthesea

In [ ]:
tag_sentence = 'Dịch vụ đặt vé trực tuyến của chúng tôi, EasyBooking, giúp bạn dễ dàng tìm kiếm, so sánh và đặt vé máy bay, khách sạn và tour du lịch. Với giao diện thân thiện và tính năng tiện lợi, chúng tôi mang đến trải nghiệm đặt vé nhanh chóng và tiết kiệm thời gian cho bạn'

In [ ]:
from underthesea import pos_tag, chunk, classify

pos_str = pos_tag(tag_sentence)
text_group = set([word for word,tag,num in chunk(tag_sentence) if tag == 'Np' or tag == 'N' or tag == 'V'])
# class_of_text = classify(tag_sentence)
nouns = set([word for word,tag in pos_str if tag == 'Np' or tag == 'N'])
print(text_group)

# Viet IDEA Spam Filtering

In [ ]:
import pandas as pd

abs_path = '/Users/macos/Code/Graduation/share-your-idea/ml-server/'
invalid_1 = pd.read_csv(abs_path + "data/random_str.csv", delimiter=',')
invalid_0 = pd.read_csv(abs_path + "data/vie/spam.csv", delimiter=';')
semantic = pd.read_csv(abs_path + "data/vie/valid_idea.csv", delimiter=';', usecols=['sentence', 'spam'])

invalid_0.head()
invalid_1.head()
semantic.head()

frames = [invalid_0, semantic, invalid_1]
 
df = pd.concat(frames)

df.sample(10)

In [ ]:
sns.countplot(df, x="spam")
plt.xlabel('Label')
plt.title('Number of ham (1) and spam (0) ideas')

In [ ]:
X = df.sentence
Y = df.spam
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words,50,input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1,name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  model.load_weights(model_weights_file)
  return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=1e-3),metrics=['accuracy'])

In [ ]:
history = model.fit(sequences_matrix,Y_train,batch_size=128,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
word_idx = 17
prediction = model.predict(test_sequences_matrix[word_idx:word_idx+5])
print(prediction, X_test[word_idx:word_idx+5])

In [ ]:
import pickle

# https://machinelearningmastery.com/save-load-keras-deep-learning-models/
# save the iris classification model as a pickle file
model_pkl_file = "../src/model/spam_filtering.pkl"  
model_json_file = "../src/model/spam_filtering.json"
model_weights_file = "../src/model/weights.h5py"


# serialize model to JSON
model_json = model.to_json()
with open(model_json_file, 'w') as file:  
  file.write(model_json)

# serialize weights to HDF5
model.save_weights(model_weights_file)

In [ ]:
from keras.models import load_model

save_model_file='../src/model/lstm'
model.save(save_model_file)

In [ ]:
def input(new_sentence, ai_model):
  new_sentence = new_sentence + "."
  test_sequences = tok.texts_to_sequences([new_sentence])
  test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=max_len)
  prediction = ai_model.predict(test_sequences_matrix)
  label = "SPAM"
  if prediction > 0.5:
    label = "VALID"
  # print(prediction, label, new_sentence[:30] + '...')
  print(test_sequences_matrix)

In [ ]:
docs = """Một ống kính độ dài 2.5cm đang chỉa thẳng xuống mặt đất để đo đạc và chụp lại""" #Spam

In [ ]:
docs = """MR.Bean là một sản phẩm tái tạo năng lượng bằng hệ thống xử lí chất thải sinh học tự động""" #Spam

In [ ]:
docs = """Agelina - đứa con của công ty mẹ Egosh, là sàn thương mại điện tử nức tiếng nước Đức""" #Spam

In [ ]:
docs = """"SmartMirror Health Monitor" là một hệ thống kết hợp gương trang điểm thông minh với chức năng theo dõi sức khỏe. Gương sẽ hiển thị thông tin về nhịp tim, nhiệt độ cơ thể và các thông số sức khỏe khác. Ngoài ra, nó cũng cung cấp các gợi ý về dinh dưỡng và chế độ tập luyện dựa trên dữ liệu cá nhân. SmartMirror Health Monitor giúp người dùng theo dõi sức khỏe một cách thuận tiện trong khi trang điểm hoặc thay đồ hàng ngày.""" #Valid

In [ ]:
docs = """"Ứng dụng kết hợp với cảm biến đất, ánh sáng và nước để giám sát và quản lý cây trồng. Người dùng sẽ nhận được thông báo để tưới nước, bón phân và chăm sóc cây đúng cách, tạo điều kiện tối ưu cho sự phát triển và sức khỏe của cây.""" #Valid

In [ ]:
docs = """Sản phẩm TechSmart đem đến cho bạn sự tiện ích và khả năng kết nối thông minh. Với tính năng đột phá và hiệu suất cao, chúng tôi giúp bạn trải nghiệm công nghệ tối ưu và tận hưởng cuộc sống hiện đại""" #Spam

In [ ]:
input(docs, model)

### Gibberish Detector

In [ ]:
!pip3 install gibberish-detector

In [ ]:
!gibberish-detector train data/WIKI.txt > vie.model

In [ ]:
from gibberish_detector import detector

# open('data/WIKI.txt')

Detector = detector.create_from_model('vie.model')
print(Detector.is_gibberish('Chào, tớ là osijd lldkn qwoie')) #Gibberish

In [ ]:
import pandas as pd
import sys
from underthesea import chunk, pos_tag
from keras.models import Sequential, model_from_json
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from gensim import models
from gibberish_detector import detector
import pickle

model_json_file = "../src/model/spam_filtering.json"
model_weights_file = "../src/model/weights.h5py"
word2vec_path='../src/model/word2vec.vie.model.bin'
gibberish_path='../src/model/gibberish.vie.model'
max_words = 1000
max_len = 150

# with open(model_json_file, 'r') as file:  
#   json_model = file.read()
#   lstm_model = model_from_json(json_model)
#   lstm_model.load_weights(model_weights_file)
#   lstm_model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=1e-3),metrics=['accuracy'])

class SpamFiltering:
  def __init__(self, lstm_model) -> None:
    self.tokenizer = Tokenizer(num_words=max_words)
    self.detector = detector.create_from_model(gibberish_path)
    self.lstm_model = lstm_model

  def is_idea_spam(self, idea_object):
    desc = [idea_object.solution + '.']
    print(desc)
    if self.detector.is_gibberish(desc):
      return "SPAM"
    self.tokenizer.fit_on_texts(desc)
    test_sequences = self.tokenizer.texts_to_sequences(desc)
    test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=max_len)
    print(test_sequences)
    prediction = self.lstm_model.predict(test_sequences_matrix)
    # prediction = model.predict(test_sequences_matrix)
    label = "SPAM"
    if prediction > 0.5:
      label = "VALID"
    elif prediction > 0.3:
      label = "WARNING"
    print(prediction, f' -> {label} :: "{desc[:30]}..."')
    return label

In [ ]:
class IdeaObj:
  def __init__(self,problem='',solution='') -> None:
    self.problem=problem
    self.solution=solution

In [ ]:
def main():
  try:
    lstm_model=None
    with open(model_json_file, 'r') as file:  
      json_model = file.read()
      lstm_model = model_from_json(json_model)
      lstm_model.load_weights(model_weights_file).expect_partial()
      # lstm_model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=1e-3),metrics=['accuracy'])
    spam = SpamFiltering(lstm_model)
    problem='tài chính'
    solution="Một ứng dụng cho phép người dùng quản lí tài chính cá nhân bằng trực quan hóa số liệu và hỗ trợ quyết định đầu tư, tiết kiệm."
    solution="""Ứng dụng kết hợp với cảm biến đất, ánh sáng và nước để giám sát và quản lý cây trồng. Người dùng sẽ nhận được thông báo để tưới nước, bón phân và chăm sóc cây đúng cách, tạo điều kiện tối ưu cho sự phát triển và sức khỏe của cây."""
    idea = IdeaObj(problem=problem, solution=solution)
    spam.is_idea_spam(idea)
  except ValueError as ve:
    return str(ve)

main()